# Institute for Behavioral Genetics International Statistical Genetics 2021 Workshop 

# Day 08 - Common Variant Analysis of Sequencing Data with Hail

Once you have reviewed the [lecture videos](https://youtu.be/2N_VqmX22Xg), you will likely be curious about the power of Hail in optimally analyzing sequencing data. 

In this practical, we will learn how to:

1) Use simple python code and Jupyter notebooks.

2) Use Hail to import a VCF and run basic queries over sequencing data.

3) Use Hail to perform some basic quality control on sequencing data.

4) Use Hail to run a basic genome-wide association study on common variants in sequencing data.


### Framing

This practical contains a lot of new material, and the goal of this workbook is not for students to be able to reproduce from memory all the various capabilities demonstrated here. Instead, the goal is for students to get a sense for the kind of analysis tasks that sequencing data requires, and to gain some exposure to what these analyses look like in Hail. 

There is no one-size-fits-all sequencing analysis pipeline, because each sequencing dataset will have unique properties that need to be understood and accounted for in QC and analysis. Hail can empower you to interrogate sequencing data, but it cannot give you all the questions to ask!

#### Phenotypes

By this point, you are in a breakout room with other students.

`sleep_duration`

`tea_intake_daily`

`general_happiness`             

`screen_time_per_day`

Your assignment would be to :

1) run through basic exploratory snippets of code

2) What is the distribution of people who have the phenotype that was assigned to your group? A simple list from `count()` or `show()` would suffice! 

3) Create a logistic/linear model with the assigned phenotype outcome using [Hail documentation](https://hail.is/docs/0.2/methods/stats.html#hail.methods.logistic_regression_rows). Use the search function at the top of the documentation page if you need more information!  

4) Print QQ plot. What does the QQ plot tell you?

5) Print Manhattan plot. Are there significant hits?

# Hail on Jupyter

Part of what we think is so exciting about Hail is that Hail development has coincided with other technological shifts in the data science community.

Five years ago, most computational biologists analyzed sequencing data using command-line tools, and took advantage of research compute clusters by explicitly using scheduling frameworks like LSF or Sun Grid Engine. These clusters are powerful resources, but it requires a great deal of extra thought and effort to manage pipelines running on them.

Today, most Hail users run Hail from within interactive Python notebooks (like this one!) backed by thousands of cores on public compute clouds like [Google Cloud](https://cloud.google.com/), [Amazon Web Services](https://aws.amazon.com/), or [Microsoft Azure](https://azure.microsoft.com/). You don't need to share a cluster with hundreds of other scientists, and you only need to pay for the resources that you use.

# 1. Using Jupyter

### Running cells
Evaluate cells using SHIFT + ENTER. Select the next cell and run it

In [ ]:
print('Hello, world')

### Modes

Jupyter has two modes, a **navigation mode** and an **editor mode**.

#### Navigation mode:

 - <font color="blue"><strong>BLUE</strong></font> cell borders
 - `UP` / `DOWN` move between cells
 - `ENTER` while a cell is selected will move to **editing mode**.
 - Many letters are keyboard shortcuts! This is a common trap.
 
#### Editor mode:

 - <font color="green"><strong>GREEN</strong></font> cell borders
 - `UP` / `DOWN`/ move within cells before moving between cells.
 - `ESC` will return to **navigation mode**.
 - `SHIFT + ENTER` will evaluate a cell and return to **navigation mode**.

### Cell types

There are several types of cells in Jupyter notebooks. The two you will see here are **Markdown** (text) and **Code**.

In [ ]:
# This is a code cell
my_variable = 5

**This is a markdown cell**, so even if something looks like code (as below), it won't get executed!

my_variable += 1

### Tips and tricks

Keyboard shortcuts:

 - `SHIFT + ENTER` to evaluate a cell
 - `ESC` to return to navigation mode
 - `y` to turn a markdown cell into code
 - `m` to turn a code cell into markdown
 - `a` to add a new cell **above** the currently selected cell
 - `b` to add a new cell **below** the currently selected cell
 - `d, d` (repeated) to delete the currently selected cell
 - `TAB` to activate code completion
 
To try this out, create a new cell below this one using `b`, and print `my_variable` by starting with `print(my` and pressing `TAB`!

## <strong style="color: red;">Resetting Jupyter if you get stuck</strong>

If at any point during this practical, you are unable to successfully run cells, it is possible that your Python interpreter is in a bad state due to cells being run in an incorrect order. If this happens, you can recover a working session by doing the following:

1. Navigate to the "Kernel" menu at the top, and select "Restart and clear output".

2. Select the cell you were working on, then select "Run all above" from the "Cell" menu at the top.


# 2. Set up our Python environment

In addition to Hail, we import a few methods from the Hail plotting library. We'll see examples soon!
For installation instructions that can help you set up Hail at home, please visit the [getting started page](https://hail.is/docs/0.2/getting_started.html).

In [ ]:
import hail as hl
from hail.plot import output_notebook, show

Now we initialize Hail and set up plotting to display inline in the notebook.

In [ ]:
hl.init()
output_notebook()

The demonstration materials are designed to work on a small (~16MB) downsampled chunk of the publically available Human Genome Diversity Project (HGDP) dataset. HGDP is a super-set of the well-known [1000 genomes](https://www.internationalgenome.org/) dataset, with a broader group of represented populations.


It is possible to call command-line utilities from Jupyter by prefixing a line with a `!`. This would access the unix-based programming language that you have been learning for the past 1.5 weeks:

In [ ]:
! ls -1 resources/

# 3. Explore genetic data with Hail

#### Learning Objectives:

- To be comfortable exploring Hail data structures.
- To understand categories of functionality for performing QC.

### Import data from VCF

The [Variant Call Format (VCF)](https://en.wikipedia.org/wiki/Variant_Call_Format) is a common file format for representing genetic data collected on multiple individuals (samples).

Hail's [import_vcf](https://hail.is/docs/0.2/methods/impex.html#hail.methods.import_vcf) imports this file to a Hail data structure representing a matrix of genetic data.

While VCF is a text format that is easy for humans to read, it is very inefficient to process on a computer. 

The first thing we do is import (`import_vcf`) and convert the `VCF` file into a Hail native file format. This is done by using the `write` method below. The resulting file is **much** faster to process.

In [ ]:
hl.import_vcf('resources/hgdp_subset.vcf.bgz', min_partitions=4, reference_genome='GRCh38')\
.write('resources/hgdp.mt', overwrite=True)

### HGDP as a Hail `MatrixTable`

We represent genetic data as a Hail [`MatrixTable`](https://hail.is/docs/0.2/overview/matrix_table.html), and name our variable `mt` to indicate this.

In [ ]:
mt = hl.read_matrix_table('resources/hgdp.mt')

### What is a `MatrixTable`?

Let's explore it!

Take your time here, about 5 minutes or so to explore every part of the widget and observe what was captured.

You can see:
 - **numeric** types:
     - integers (`int32`, `int64`), e.g. `5`
     - floating point numbers (`float32`, `float64`), e.g. `5.5` or `3e-8`
 - **strings** (`str`), e.g. `"Foo"`
 - **boolean** values  (`bool`) e.g. `True`
 - **collections**:
     - arrays (`array`), e.g. `[1,1,2,3]`
     - sets (`set`), e.g. `{1,3}`
     - dictionaries (`dict`), e.g. `{'Foo': 5, 'Bar': 10}`
 - **genetic data types**:
     - loci (`locus`), e.g. `[GRCh37] 1:10000` or `[GRCh38] chr1:10024`
     - genotype calls (`call`), e.g. `0/2` or `1|0`

### Exercise
Take a few moments to explore the interactive representation of the matrix table below.

* Where is the variant information (`locus` and `alleles`)? 
* Where is the sample identifier (`s`)?
* Where is the genotype quality `GQ`?

In [ ]:
mt.describe(widget=True)

### `show`

Hail has a variety of functionality to help you quickly interrogate a dataset. The `show()` method prints the first few values of any field, and even prints in pretty HTML output in a Jupyter notebook! 

In [ ]:
mt.s.show()

It is also possible to show() the matrix table itself, which prints a portion of the top-left corner of the variant-by-sample matrix:

In [ ]:
mt.show()

The above output is visually noisy because the matrix table has as lot of information in it. `show`ing just the called genotype (`GT`) is a bit more friendly.

The printed representation of GT is explained below, where `a` is the reference allele and `A` is the alternate allele:

`0/0` : homozygous reference or `aa`

`0/1` : heterozygous or `Aa`

`1/1` : homozygous alternate or `AA` 


In [ ]:
mt.GT.show()

### `summarize`
`summarize` Prints (potentially) useful information about any field or object:

`DP` is the read depth (number of short reads spanning a position for a given sample).

In [ ]:
mt.DP.summarize()

`AD` is the array of allelic depth per allele at a called genotype. Note especially the missingness properties:

In [ ]:
mt.AD.summarize()

### Exercise

In the empty cell below, summarize some of the other fields on the matrix table. You can use the interactive widget above to find the names of some of the other fields.

Share any interesting findings with your colleagues!

### `count`

`MatrixTable.count` returns a tuple with the number of rows (variants) and number of columns (samples).

In [ ]:
mt.count()

The count above tells us that we have 10,441 variants and 392 samples. This is just a tiny slice of a real sequencing dataset, which today can include more than 1 billion rows and more than 100,000 samples.

### Hail has functions built for genetics

For example, `hl.summarize_variants` prints useful statistics about the genetic variants in the dataset. These are not part of the generic `summarize()` function, which must support all kinds of data, not just variant data!

To understand some of the output from this, let's attempt to figure out line by line?

A **variant** is when you have an alternate form your reference allele

When you would see `Alleles per variant`, observe that the number of allele is 2 indicating that your dataset is made up only of dimorphic alleles.

A **SNP** is a single nucleotide polymorphism where this is a variant type where only a single nucleotide change has occured.

**Transition** refers to a point mutation in which one base is replaced by another of the same class (purine or pyrimidine) while **transversion** refers to a single nucleotide change in which a purine is replaced with a pyrimidine or vice versa. 

*Purines = `A` and `G` nucleotides*

*Pyramidines = `C` and `T` nucleotides*

**Transition Transversion Ratio**: Across the entire genome the ratio of transitions to transversions is typically around 2. In protein coding regions, this ratio is typically higher, often a little above 3 (transversions are likely to change the underlying amino acid).

A **contig** in this context would be a chromosome. In this dataset, you have autosomal and sex chromosomes included.

In [ ]:
hl.summarize_variants(mt)

# Annotation and quality control

## Integrate sample information

Your dataset currently only has sample IDs and genetic data. In order to run a toy GWAS, we would need phenotype information which we can easily add an annotation to your `matrixTable`.

We will be using the following text file containing phenotype information:

In [ ]:
! head resources/HGDP_sample_data.tsv

We can import it as a [Hail Table](https://hail.is/docs/0.2/overview/table.html) with [hl.import_table](https://hail.is/docs/0.2/methods/impex.html?highlight=import_table#hail.methods.import_table).

We call it `sd` for "sample data".

In [ ]:
sd = hl.import_table('resources/HGDP_sample_data.tsv',
                     key='sample_id',
                     impute=True)

_To note_:

The "key" argument tells Hail to use the `sample_id` field as the table key, which is used to find matching values in  joins. In a moment, we will be joining the `sd` table onto our matrix table so that we can use the sample data fields in our QC and analysis. It is also possible to specify a new key for an existing table using the `.key_by(...)` method.

The "impute" argument tells Hail to impute the data types of the fields on the table. What does this mean? It means that you can ask Hail to figure out what is the data type in each column field such as `str` (string or just characters), `bool` (boolean or just true and false), `float64` (float or numbers with decimals), or `int32` (integer or numbers without decimals/whole numbers). If you don't use the `impute` flag or specify types manually with the `types` argument, each field will be imported as a string.

While we can see the names and types of fields in the logging messages, we can also `show` this table:

In [ ]:
sd.show()

And we can `summarize` each field in `sd`:

In [ ]:
sd.summarize()

## Add sample data into our HGDP `MatrixTable`

Let's now merge our genetic data (`mt`) with our sample data (`sd`).

It just takes one line:

In [ ]:
mt = mt.annotate_cols(sample_data = sd[mt.s])

### What's going on here?

Understanding what's going on here is a bit more difficult. To understand, we need to understand a few pieces:

#### 1. `annotate` methods

In Hail, `annotate` methods refer to **adding new fields**. 

 - `MatrixTable`'s `annotate_cols` adds new column (**sample**) fields.
 - `MatrixTable`'s `annotate_rows` adds new row (**variant**) fields.
 - `MatrixTable`'s `annotate_entries` adds new entry (**genotype**) fields.
 - `Table`'s `annotate` adds new row fields.

In the above cell, we are adding a new column (**sample**) field called "sample_data". This field should be the values in our table `sd` associated with the sample ID `s` in our `MatrixTable` - that is, this is performing a **join**.

Python uses square brackets to look up values in dictionaries:

    >>> d = {'foo': 5, 'bar': 10}
    
    >>> d['foo']
    'bar'

You should think of this in much the same way - for each column of `mt`, we are looking up the fields in `sd` using the sample ID `s`.

Let's see how the matrix table has changed:

In [ ]:
mt.describe(widget=True)

## Query phenotype fields

Despite a particular focus on sequencing data, Hail is a general-purpose data exploration tool. We will use this general-purpose functionality to answer some questions about the sample data we have imported.

The code below uses the `aggregate_cols` method on our matrix table, which computes aggregate statistics about column (sample) data. There are also methods for `aggregate_rows` (aggregate over row data) and `aggregate_entries` aggregate over all of the entries in our matrix, one per variant per sample).

Hail **aggregators** can be recognized by the `hl.agg` prefix. Some examples:

 - `hl.agg.fraction(CONDITION)` - compute the fraction of values at which `CONDITION` is true.
 - `hl.agg.count_where(CONDITION)` - compute the number of values at which `CONDITION` is true.
 - `hl.agg.stats(X)` - compute a few useful statistics about `X`.
 - `hl.agg.counter(X)` - compute the number of occurrences of each unique value of `X`. Useful for categorical fields like strings, not as useful for numbers!
 - `hl.agg.corr(X, Y)` - compute the Pearson correlation coefficient between X and Y values.
 - For more adventurous students, see the [full list of aggregators](https://hail.is/docs/0.2/aggregators.html#sec-aggregators).

To start, we will compute the occurrences of each value of `sex_karyotype`:

For instance: `mt.aggregate_cols(hl.agg.fraction(mt.pheno.phenotype))` would now be `mt.aggregate_cols(hl.agg.fraction(mt.pheno.dog_person))`

To recap, the phenotypes are: 

 - `sleep_duration`
 - `tea_intake_daily`
 - `general_happiness`
 - `screen_time_per_day`


In [ ]:
mt.aggregate_cols(hl.agg.counter(mt.sample_data.sex_karyotype))

The above result tells us that slightly more than half of our samples are XY, and the rest are XX. What should you do if some of your samples are neither XX or XY? That depends on the analysis you are trying to do, but you should be ready to think about this case!

How many people are in each self-reported major continental ancestry group?

In [ ]:
mt.aggregate_cols(hl.agg.counter(mt.sample_data.continental_pop))

### Exercise

Try changing `continental_pop` to `pop` and rerunning the cell above. Most of the populations are abbreviated, but see if you can find an ancestral population from each continent among the non-abbreviated ones!

The numeric aggregators look similar:

In [ ]:
mt.aggregate_cols(hl.agg.stats(mt.sample_data.sleep_duration))

### Exercise

Use the `fraction` and `count_where` aggregators defined above to answer the following questions in the cells below:

1. How many samples drink more than 8 cups of tea per day? *Hint: the CONDITION will take the form `SOMETHING > 8`.*

2. What fraction of samples sleep less than 4 hours per day?


### More complicated aggregators

You've seen and tried a taste of aggregator functionality, but aggregators in Hail can be very expressive. As an example, we're going to compute the sample IDs of the five happiest XX individuals. Don't worry about being able to reproduce this on your own right now!


In [ ]:
mt.aggregate_cols(
    hl.agg.filter(
        mt.sample_data.sex_karyotype == 'XX',
        hl.agg.take(hl.struct(sample_id=mt.s, happiness=mt.sample_data.general_happiness),
                    5,
                    ordering=-mt.sample_data.general_happiness)
    )
)

## Sample QC

Something you have learned in your QC section is: _garbage in, garbage out_. Good QC takes time and thoughtfulness but is necessary for robust results. 

Here, we run through some simple sample qc steps, but **these steps are not a one-size-fits-all solution for QC on your own data!**

Hail has the function [hl.sample_qc](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.sample_qc) to compute a list of useful statistics about samples from sequencing data. This function adds a new column field, `sample_qc`, with the computed statistics. Note that this doesn't actually remove samples for you -- those decisions are up to you. The `sample_qc` method gives you some data you can use as a starting point.

**Click the link** above to see the documentation, which lists the fields and their descriptions.

In [ ]:
mt = hl.sample_qc(mt)

In [ ]:
mt.describe(widget=True)

Hail includes a plotting library built on [bokeh](https://bokeh.pydata.org/en/latest/index.html) that makes it easy to visualize fields of Hail tables and matrix tables.

Let's visualize the distribution of `Mean DP` (`DP` = Read Depth) to `Call Rate`.

Note that you can **hover over points with your mouse to see the sample IDs!**

In [ ]:
p = hl.plot.scatter(x=mt.sample_qc.dp_stats.mean,
                    y=mt.sample_qc.call_rate,
                    xlabel='Mean DP',
                    ylabel='Call Rate',
                    hover_fields={'ID': mt.s},
                    size=8)
show(p)

### Exercise

If you are comfortable in Python, try adding the following argument into the plot function argument list above:

    label=mt.sample_data.sex_karyotype,

### Filter columns using generated QC statistics

Before filtering samples, we should compute a raw sample count:

In [ ]:
mt.count_cols()

`filter_cols` removes entire columns from the matrix table. Here, we keep columns (samples) where the `call_rate` is over 92%:

In [ ]:
mt = mt.filter_cols(mt.sample_qc.call_rate >= 0.92)

We can compute a final sample count:

In [ ]:
mt.count_cols()

How many samples did not meet your QC criteria?

In [ ]:
p = hl.plot.histogram(mt.DP, legend='DP', title='DP Histogram', range=(0, 100))
show(p)


## Variant QC

Now that we have successfully gone through basic sample QC using the `Hail` function called `sample_qc`, let's run variant QC.


Hail has the function [hl.variant_qc](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.variant_qc) to compute a list of useful statistics about **variants** from sequencing data.

Once again, **Click the link** above to see the documentation!

In [ ]:
mt = hl.variant_qc(mt)

In [ ]:
mt.describe(widget=True)

Find the `variant_qc` output!

We can `show()` the computed information:

In [ ]:
mt.variant_qc.show()

Metrics like `call_rate` are important for QC. Let's plot the cumulative density function of call rate per variant:

In [ ]:
show(hl.plot.cdf(mt.variant_qc.call_rate))

Before filtering variants, we should compute a raw variant count:

In [ ]:
# pre-qc variant count
mt.count_rows()

`filter_rows` removes entire rows of the matrix table. Here, we keep rows where the `call_rate` is over 95%:

In [ ]:
mt = mt.filter_rows(mt.variant_qc.call_rate > 0.95)

After filtering, we can see more resolution of the top end of the call rate distribution:

In [ ]:
show(hl.plot.cdf(mt.variant_qc.call_rate))

We can then compute the final sample and variant count:

In [ ]:
mt.count()

How many variants did you lose from your variant QC? 

# Association Testing and PCA

As with every GWAS, we want to find an association of a trait to locus/loci of interest. 

Before that, we first filter to common variants (those with an alternate allele frequency over 1%). Generally, GWAS cannot detect signal from extremely rare variants, like those only observed in one individual.

In [ ]:
base = mt

In [ ]:
mt = base.filter_rows(hl.agg.call_stats(base.GT, base.alleles).AF[1] > 0.01)

In [ ]:
mt.count()

How many variants did you lose from your common variant filter? 

Remember that in a GWAS we're running independent association tests on each variant.

In Hail, the method we use is [hl.linear_regression_rows](https://hail.is/docs/0.2/methods/stats.html#hail.methods.linear_regression_rows). Why isn't this called `hl.gwas`? Modularity! There are applications for this statistical method other than genome wide association studies.

We will start by using the `sleep_duration` phenotype. At the end, you will have a chance to try the others.

In [ ]:
phenotype = 'sleep_duration'

In [ ]:
gwas = hl.linear_regression_rows(y=mt.sample_data[phenotype], 
                                 x=mt.GT.n_alt_alleles(), 
                                 covariates=[1.0]  # intercept term
                                )

The resulting output from the line above is a `table`. Let's look at what the table looks like.

Out of all of the fields, we would recommend focusing your understanding of the p-value and beta effect when determining if you have a GWAS signal.

**However**, keep in mind that the results thus far may need your model to be adjusted.

In [ ]:
gwas.show()

## Visualization

Let’s visualize our association test results from the linear regression. We can do so by creating 2 common plots: a [Manhattan plot](https://en.wikipedia.org/wiki/Manhattan_plot) and a [Q-Q plot](https://en.wikipedia.org/wiki/Q%E2%80%93Q_plot).

We'll start with the Manhattan plot:

In [ ]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

### Manhattan plot interpretation

The dashed red line is the line for genome-wide significance with a typical Bonferroni correction -- the line is at 5e-8.

We have several points above the line -- mouse over to see the loci they refer to. This means we're ready to publish our sleep GWAS in *Nature*, right?


...right?

### Q-Q plot
The plot that accompanies a Manhattan plot is the Q-Q (quantile-quantile) plot.

In [ ]:
p = hl.plot.qq(gwas.p_value)
show(p)

The Q-Q plot can clearly indicate widespread inflation or deflation of p-values. Is either of those properties present here?

**Is this GWAS well controlled? Discuss with your group.**

Wikipedia has a good description of [genomic control estimation](https://en.wikipedia.org/wiki/Genomic_control) (lambda GC) to read later.

## Confounded! PCA to the rescue.

If you've done a GWAS before, you've probably included a few other covariates that might be confounders -- age, sex, and principal components.

Principal components are a measure of genetic ancestry, and can be used to control for [population stratification](https://en.wikipedia.org/wiki/Population_stratification).

Principal component analysis (PCA) is a very general statistical method for reducing high dimensional data to a small number of dimensions which capture most of the variation in the data. Hail has the function [pca](https://hail.is/docs/0.2/methods/stats.html#hail.methods.pca) for performing generic PCA.

PCA typically works best on normalized data (e.g. mean centered). Hail provides the specialized function [hwe_normalized_pca](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.hwe_normalized_pca) which first normalizes the genotypes according to the Hardy-Weinberg Equilibium model.

In [ ]:
pca_eigenvalues, pca_scores, pca_loadings = hl.hwe_normalized_pca(mt.GT, compute_loadings=True)

In [ ]:
pca_scores = pca_scores.checkpoint('resources/pca_scores.ht', overwrite=True)

The pca function returns three things:
* **eigenvalues**: an array of doubles
* **scores**: a table keyed by sample
* **loadings**: a table keyed by variant

The **loadings** are the *principal directions*, each of which is a weighted sum of variants (a "virtual variant"). By default, `hwe_normalized_pca` gives us 10 principal directions.

In [ ]:
mt = mt.annotate_cols(pca = pca_scores[mt.s])

## Control confounders and run another GWAS

Now that we have computed principal components and saved it into our `mt`, let’s attempt to adjust the inflation that we saw in our initial Q-Q plot e.g.  `mt.pheno.is_female`. We will now add PCs as `covariates` in `linear_regression_rows`:


In [ ]:
gwas2 = hl.linear_regression_rows(
    y=mt.sample_data[phenotype], 
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pca.scores[0], mt.pca.scores[1], mt.pca.scores[2]])

In [ ]:
p = hl.plot.qq(gwas2.p_value)
show(p)

The above Q-Q plot indicates a much better-controlled GWAS. Let's try the Manhattan plot:

In [ ]:
p = hl.plot.manhattan(gwas2.p_value)
show(p)

### Are there any significant hits? Do you trust these results? Why or why not?

Discuss with your group - roughly how many samples would you need to discover rare variant signal? polygenic signal?

### Exercise

Go back to the "Association Testing and PCA" section above and rerun the rest of the notebook with a different phenotype.

### Exercise

Change the "gwas2" cell to experiment with how many principal components are needed as covariates to properly control this GWAS. How many are needed here in our tiny simulated example? How many are needed in a typical GWAS?

# Return to main room for discussion